In [1]:
import pandas as pd
import numpy as np
import json
import requests

from keplergl import KeplerGl

In [2]:
# Getting the data to Pandas dataframe

filename = 'gps-trace.json'

df = pd.read_json(filename)
df = df['points']
df = pd.json_normalize(df)
df = df[['gps_time', 'longitude', 'latitude']]
df.head()

,gps_time,longitude,latitude
0,2020-07-31T19:05:46,67.153923,54.591755
1,2020-07-31T19:04:34,67.154312,54.591812
2,2020-07-31T19:04:27,67.154930,54.591877
3,2020-07-31T19:04:07,67.156960,54.591896
4,2020-07-31T19:03:48,67.158783,54.591942


In [3]:
# Adding some noise to the trace to make it more difficult

mu, sigma = 0, 0.0002 
noise = np.random.normal(mu, sigma, [len(df.index),])
df['latitude'] = df['latitude'] + noise
df['longitude'] = df['longitude'] + noise

In [4]:
# Plotting the data on the map and saving it to the html file

title = 'Non-optimized.html'

map = KeplerGl(height=600, width=600)
# add data to keplergl map
map.add_data(data=df, name=title)
map.save_to_html(file_name=title)
map

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter
Map saved to Non-optimized.html!


KeplerGl(data={'Non-optimized.html': {'index': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, …

In [5]:
df['gps_time'] = pd.to_datetime(df['gps_time'])
df = df.sort_values(by=['gps_time'])

In [6]:
# Optimizing data to get it to Valhalla's Meili

df_trip_for_meili = df[['longitude', 'latitude', 'gps_time']].copy()
df_trip_for_meili.columns = ['lon', 'lat', 'time']
df_trip_for_meili.head()

,lon,lat,time
81,67.147783,54.669725,2020-07-31 18:52:49
80,67.147004,54.669144,2020-07-31 18:52:53
79,67.147285,54.669464,2020-07-31 18:52:55
78,67.147856,54.670005,2020-07-31 18:52:57
77,67.147519,54.669461,2020-07-31 18:53:02


In [7]:
# Preparing the request to Valhalla's Meili

meili_coordinates = df_trip_for_meili.to_json(orient='records')
meili_head = '{"shape":'
meili_tail = ""","search_radius": 150, "shape_match":"map_snap", "costing":"auto", "format":"osrm"}"""
meili_request_body = meili_head + meili_coordinates + meili_tail

In [8]:
# Sending a request to Valhalla's Meili

url = "http://localhost:8002/trace_route"
headers = {'Content-type': 'application/json'}
data = str(meili_request_body)

r = requests.post(url, data=data, headers=headers)

In [9]:
# Parsing the response from Valhalla's Meili

if r.status_code == 200:
    
        response_text = json.loads(r.text)

        resp = str(response_text['tracepoints'])

        resp = resp.replace("'waypoint_index': None", "'waypoint_index': '#'")
        resp = resp.replace("None", "{'matchings_index': '#', 'name': '', 'waypoint_index': '#', 'alternatives_count': 0, 'distance': 0, 'location': [0.0, 0.0]}")

        resp = resp.replace("'", '"')

        resp = json.dumps(resp)
        resp = json.loads(resp)
        
        df_response = pd.read_json(resp)
        df_response = df_response[['name', 'distance', 'location']]
                
        df_trip_optimized = pd.merge(df_trip_for_meili, df_response, left_index=True, right_index=True)

In [10]:
df_trip_optimized.head()

,lon,lat,time,name,distance,location
81,67.147783,54.669725,2020-07-31 18:52:49,,17.835,"[67.153885, 54.591877]"
80,67.147004,54.669144,2020-07-31 18:52:53,,7.219,"[67.154449, 54.591881]"
79,67.147285,54.669464,2020-07-31 18:52:55,,35.674,"[67.15461, 54.591881]"
78,67.147856,54.670005,2020-07-31 18:52:57,,17.842,"[67.15715, 54.591915]"
77,67.147519,54.669461,2020-07-31 18:53:02,,7.660,"[67.158714, 54.591949]"


In [11]:
# Parsing the location column of Meili's response

for i, row in df_trip_optimized.iterrows():
        df_trip_optimized.at[i, 'longitude'] = df_trip_optimized.at[i,'location'][0]
        df_trip_optimized.at[i, 'latitude'] = df_trip_optimized.at[i,'location'][1]
        
        #saving all the geopoints that couldn't be snapped to road
        if df_trip_optimized.at[i, 'longitude'] == 0.0:
            df_trip_optimized.at[i, 'longitude'] = df_trip_optimized.at[i,'lon']
            df_trip_optimized.at[i, 'latitude'] = df_trip_optimized.at[i,'lat']
            
df_trip_optimized = df_trip_optimized.drop(['location', 'lon', 'lat'], 1)

In [12]:
df_trip_optimized.head()

,time,name,distance,longitude,latitude
81,2020-07-31 18:52:49,,17.835,67.153885,54.591877
80,2020-07-31 18:52:53,,7.219,67.154449,54.591881
79,2020-07-31 18:52:55,,35.674,67.154610,54.591881
78,2020-07-31 18:52:57,,17.842,67.157150,54.591915
77,2020-07-31 18:53:02,,7.660,67.158714,54.591949


In [13]:
df_trip_optimized['time'] = df_trip_optimized['time'].astype(str)

In [14]:
# Plotting the data on the map and saving it to the html file

title = 'Optimized.html'

map = KeplerGl(height=600, width=600)
# add data to keplergl map
map.add_data(data=df_trip_optimized, name=title)
map.save_to_html(file_name=title)
map

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter
Map saved to Optimized.html!


KeplerGl(data={'Optimized.html': {'index': [81, 80, 79, 78, 77, 76, 75, 74, 73, 72, 71, 70, 69, 68, 67, 66, 65…